In [37]:
import os
import sys
import pathlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use("ggplot")

data_path = pathlib.Path("/devcode/GATE-private/notebooks/gate-results.csv")

In [38]:
df = pd.read_csv(data_path)

# Print the first few rows of the DataFrame to see what's inside
print(df.head())

Name  Runtime Notes     State  Tags   
0     "athena-svhn-talip-base16-witav-42"     4173     -  finished   NaN  \
1   "athena-svhn-laion-vit-base16-224-42"     2685     -  finished   NaN   
2   "athena-svhn-talip-base16-witav-2306"     3414     -  finished   NaN   
3  "athena-svhn-clip-vit-base16-224-1337"     6400     -    failed   NaN   
4   "athena-svhn-talip-base16-witav-1337"     3963     -  finished   NaN   

   dataloader.num_workers                                   dataset._target_   
0                    16.0  gate.data.image.classification.svhn.build_gate...  \
1                    64.0  gate.data.image.classification.svhn.build_gate...   
2                    64.0  gate.data.image.classification.svhn.build_gate...   
3                    40.0  gate.data.image.classification.svhn.build_gate...   
4                    16.0  gate.data.image.classification.svhn.build_gate...   

   dataset.num_classes  init_global_step   
0                 10.0               0.0  \
1                 10.0               0.0   
2                 10.0               0.0   
3                 10.0            4500.5   
4                 10.0               0.0   

                                      model._target_  ...   
0  gate.models.task_specific_models.classificatio...  ...  \
1  gate.models.task_specific_models.classificatio...  ...   
2  gate.models.task_specific_models.classificatio...  ...   
3  gate.models.task_specific_models.classificatio...  ...   
4  gate.models.task_specific_models.classificatio...  ...   

  validation/accuracy_top_1-epoch-mean validation/accuracy_top_1-epoch-std   
0                            99.407326                  0.6912270784378052  \
1                            99.407326                  0.7215445637702942   
2                            99.515083                  0.7291801571846008   
3                            99.602638                  0.6229731142520905   
4                            99.555496                  0.6397664546966553   

  validation/accuracy_top_5 validation/accuracy_top_5-epoch-mean   
0                     100.0                            99.878769  \
1                     100.0                            99.905708   
2                     100.0                            99.905708   
3                     100.0                            99.905708   
4                     100.0                            99.892242   

   validation/accuracy_top_5-epoch-std validation/logits-epoch-mean   
0                   0.2853368818759918                    -0.004663  \
1                   0.2955079674720764                    -0.043949   
2                   0.2567276954650879                    -0.095437   
3                   0.2567276954650879                    -0.000746   
4                   0.3086479902267456                    -0.010631   

  validation/logits-epoch-std  validation/loss  validation/loss-epoch-mean   
0         0.01228101085871458         0.000664                    0.039007  \
1         0.02246733382344246         0.000062                    0.035251   
2         0.01151279266923666         0.000197                    0.027152   
3        0.013235040009021757         0.000184                    0.028256   
4        0.013362712226808071         0.000249                    0.028540   

   validation/loss-epoch-std  
0        0.04896154627203941  
1        0.05310678482055664  
2        0.04407551139593125  
3       0.043903104960918427  
4        0.04779575020074845  

[5 rows x 45 columns]

In [3]:
# Print the summary statistics of the DataFrame
print(df.describe())

       testing/accuracy_top_1-epoch-mean  testing/accuracy_top_1-epoch-std   
count                         130.000000                        130.000000  \
mean                           65.825859                          4.967498   
std                            27.083131                          3.582391   
min                             2.274525                          1.162434   
25%                            56.174597                          3.216839   
50%                            77.484615                          3.790733   
75%                            82.613693                          4.446880   
max                            96.229073                         15.894830   

       testing/accuracy_top_5-epoch-mean  testing/accuracy_top_5-epoch-std   
count                         130.000000                        130.000000  \
mean                           82.054007                          3.441022   
std                            24.640253                       

In [39]:
import pandas as pd
from rich.table import Table
from rich.console import Console


def load_and_process_data(data_path: str):
    # Load the data
    df = pd.read_csv(data_path)

    # Extract the seed number and remove it from the "Name"
    df["Seed"] = df["Name"].apply(
        lambda x: int(x.split("-")[-1].replace('"', ""))
        if x.split("-")[-1].replace('"', "").isdigit()
        else None
    )
    df["Name"] = df["Name"].apply(
        lambda x: "-".join(x.split("-")[:-1]).replace('"', "")
    )

    # Filter the DataFrame
    df = df[df["Name"].str.startswith("athena")]
    df = df[~df["Name"].str.contains("tali")]

    # df = df.dropna()

    # Extract the task name
    df["Task"] = df["Name"].apply(lambda x: x.split("-")[1])
    df["Name"] = df["Name"].apply(lambda x: "-".join(x.split("-")[2:-1]))

    # Convert columns to numeric type
    numeric_columns = [
        "validation/accuracy_top_1-epoch-mean",
        "validation/accuracy_top_5-epoch-mean",
    ]

    for col in numeric_columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    # Group by 'Name' and 'Task' and calculate mean and standard deviation
    grouped = df.groupby(["Task", "Name"])[numeric_columns].agg(
        ["mean", "std", "count"]
    )

    # Sort by 'Task'
    grouped = grouped.sort_values(by="Task")

    return grouped


df = load_and_process_data(data_path)
print(df.to_csv("data.csv"))

None

In [34]:
from rich.console import Console
from rich.table import Table

console = Console(width=100)


def print_rich_table(df: pd.DataFrame, task: str):
    # Create a new table
    table = Table(show_header=True, header_style="bold magenta")

    # Add columns
    table.add_column("Name")
    for col in df.columns:
        table.add_column(str(col), width=50)

    # Add rows
    for index, row in df.iterrows():
        table.add_row(str(index), *[str(v) for v in row])

    # Print the table
    console.print(f"Task: {task}")
    console.print(table, width=100)


# Pivot the data
for task in df.index.get_level_values("Task").unique():
    pivot_df = df.loc[task].pivot_table(
        index="Name",
        values=[
            "validation/accuracy_top_1-epoch-mean",
            "validation/accuracy_top_5-epoch-mean",
        ],
        aggfunc=["mean", "std", "count"],
    )
    # Print the table
    print_rich_table(pivot_df, task)

Task: c100

┏┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃┃ ('validation… ┃ ('validation… ┃ ('validation/… ┃ ('validation… ┃ ('validation/… ┃ ('validation… ┃
┃┃ 'count')      ┃ 'mean')       ┃ 'std')         ┃ 'count')      ┃ 'mean')        ┃ 'std')        ┃
┡╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
││ 3.0           │ 55.050549825… │ 0.23622645015… │ 3.0           │ 80.5736821492… │ 0.8293499782… │
││ 3.0           │ 72.909772237… │ 0.75360886887… │ 3.0           │ 92.0867818196… │ 0.2284887658… │
││ 3.0           │ 66.598883310… │ 1.68363837695… │ 3.0           │ 89.5998026529… │ 1.0626027462… │
││ 3.0           │ 63.666512807… │ 0.40815487583… │ 3.0           │ 87.3307291666… │ 0.6078421009… │
││ 3.0           │ 27.798253395… │ 15.0962063097… │ 3.0           │ 51.2244179301… │ 21.361073337… │
││ 3.0           │ 71.305910746… │ 2.55861859879… │ 3.0           │ 91.6551767985… │ 1.1856509731… │
││ 3.0           │ 72.223114013… │ 2.04500675116… │ 3.0           │ 92.0186131795… │ 1.1454854718… │
││ 3.0           │ 62.788756052… │ 0.51435090144… │ 3.0           │ 87.5268071492… │ 0.5274142731… │
││ 3.0           │ 65.080677032… │ 2.96661209102… │ 3.0           │ 88.2342715793… │ 1.0653565137… │
││ 3.0           │ 51.865809122… │ 0.15118458333… │ 3.0           │ 77.7941157023… │ 0.7619220914… │
││ 3.0           │ 37.440384970… │ 18.1543922817… │ 3.0           │ 61.0849412812… │ 26.635458858… │
││ 3.0           │ 70.536150614… │ 1.58090177877… │ 3.0           │ 90.3844985961… │ 1.0651663601… │
││ 3.0           │ 24.835324605… │ 0.59628645583… │ 3.0           │ 46.0523897806… │ 0.1898016810… │
└┴───────────────┴───────────────┴────────────────┴───────────────┴────────────────┴───────────────┘

Task: clvr

┏┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃┃ ('validation… ┃ ('validation… ┃ ('validation/… ┃ ('validation… ┃ ('validation/… ┃ ('validation… ┃
┃┃ 'count')      ┃ 'mean')       ┃ 'std')         ┃ 'count')      ┃ 'mean')        ┃ 'std')        ┃
┡╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
││ 3.0           │ 54.713666280… │ 0.36794319543… │ 3.0           │ 99.4462381998… │ 0.0268674263… │
││ 3.0           │ 53.538756052… │ 0.24445255470… │ 0.0           │ nan            │ nan           │
││ 3.0           │ 56.283620198… │ 2.35801065994… │ 3.0           │ 99.4266408284… │ 0.0332147549… │
││ 1.0           │ 54.839569091… │ nan            │ 1.0           │ 99.2756423950… │ nan           │
││ 2.0           │ 54.515863418… │ 0.13229929465… │ 2.0           │ 99.4196624755… │ 0.0063874391… │
││ 2.0           │ 54.115459442… │ 0.11849131272… │ 2.0           │ 99.3943061828… │ 0.0301299390… │
└┴───────────────┴───────────────┴────────────────┴───────────────┴────────────────┴───────────────┘

Task: f101

...

┏┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃┃ ('validation… ┃ ('validation… ┃ ('validation/… ┃ ('validation… ┃ ('validation/… ┃ ('validation… ┃
┃┃ 'count')      ┃ 'mean')       ┃ 'std')         ┃ 'count')      ┃ 'mean')        ┃ 'std')        ┃
┡╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
││ 2.0           │ 37.397850990… │ 17.1194416018… │ 2.0           │ 61.2371078431… │ 27.911380756… │
││ 3.0           │ 56.917188008… │ 0.08307302989… │ 3.0           │ 86.7546869913… │ 0.0614937653… │
││ 1.0           │ 46.09765625   │ nan            │ 1.0           │ 78.0890655517… │ nan           │
││ 3.0           │ 53.388020833… │ 0.26950464237… │ 3.0           │ 84.1601587931… │ 0.1666875691… │
││ 0.0           │ nan           │ nan            │ 0.0           │ nan            │ nan           │
││ 3.0           │ 52.001823425… │ 3.19403768759… │ 3.0           │ 83.1333338419… │ 2.5847618533… │
││ 3.0           │ 56.981510162… │ 0.14570601651… │ 3.0           │ 86.8052088419… │ 0.0875599655… │
││ 3.0           │ 52.899478912… │ 0.65693960408… │ 3.0           │ 83.8195292154… │ 0.4148923137… │
││ 3.0           │ 47.639321645… │ 1.17272069729… │ 3.0           │ 79.0809911092… │ 1.0119819703… │
││ 2.0           │ 37.896484375  │ 1.65783178766… │ 2.0           │ 69.9308586120… │ 1.8942695386… │
││ 2.0           │ 56.773826599… │ 0.18506310288… │ 2.0           │ 86.5910186767… │ 0.0138106793… │
└┴───────────────┴───────────────┴────────────────┴───────────────┴────────────────┴───────────────┘

Task: stl10

┏┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃┃ ('validation… ┃ ('validation… ┃ ('validation/… ┃ ('validation… ┃ ('validation/… ┃ ('validation… ┃
┃┃ 'count')      ┃ 'mean')       ┃ 'std')         ┃ 'count')      ┃ 'mean')        ┃ 'std')        ┃
┡╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
││ 3.0           │ 93.877962748… │ 0.29763216692… │ 3.0           │ 99.7395833333… │ 0.1127637244… │
││ 3.0           │ 97.434000651… │ 0.72665085135… │ 3.0           │ 99.8697916666… │ 0.1127637244… │
││ 3.0           │ 97.766255696… │ 0.29235581388… │ 3.0           │ 99.8046875     │ 0.0           │
││ 3.0           │ 94.234911600… │ 0.37383187049… │ 3.0           │ 99.7395833333… │ 0.2983447718… │
││ 3.0           │ 86.907328287… │ 0.34056441928… │ 3.0           │ 99.1267089843… │ 0.2963619001… │
││ 3.0           │ 98.4375       │ 0.33829117335… │ 3.0           │ 99.8046875     │ 0.0           │
││ 3.0           │ 95.629038492… │ 0.17549424058… │ 3.0           │ 99.8046875     │ 0.1953125     │
││ 3.0           │ 89.634969075… │ 0.93058912025… │ 3.0           │ 99.4140625     │ 0.1953125     │
││ 3.0           │ 83.113324483… │ 17.1459903125… │ 3.0           │ 97.4070587158… │ 3.6289384822… │
││ 3.0           │ 88.319414774… │ 0.61639605618… │ 3.0           │ 98.8079198201… │ 0.5292376867… │
││ 3.0           │ 79.034212748… │ 2.94576665256… │ 3.0           │ 98.0715713500… │ 0.4686620512… │
││ 3.0           │ 96.313758850… │ 0.69478779466… │ 3.0           │ 99.9348958333… │ 0.1127637244… │
││ 3.0           │ 44.498697916… │ 0.13216354592… │ 3.0           │ 85.8218841552… │ 1.6862895530… │
└┴───────────────┴───────────────┴────────────────┴───────────────┴────────────────┴───────────────┘

Task: svhn

┏┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃┃ ('validation… ┃ ('validation… ┃ ('validation/… ┃ ('validation… ┃ ('validation/… ┃ ('validation… ┃
┃┃ 'count')      ┃ 'mean')       ┃ 'std')         ┃ 'count')      ┃ 'mean')        ┃ 'std')        ┃
┡╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
││ 3.0           │ 98.868532816… │ 0.16877663437… │ 3.0           │ 99.8069330851… │ 0.0741834542… │
││ 3.0           │ 99.317529042… │ 0.25202566670… │ 3.0           │ 99.9102007548… │ 0.0077778340… │
││ 3.0           │ 99.250179290… │ 0.24107871176… │ 3.0           │ 99.8877487182… │ 0.0411506011… │
││ 3.0           │ 99.030171712… │ 0.65546808909… │ 3.0           │ 99.9012196858… │ 0.0664457216… │
││ 3.0           │ 87.871765136… │ 16.7365555401… │ 3.0           │ 98.3375218709… │ 2.2624497379… │
││ 2.0           │ 99.353448867… │ 0.01905172422… │ 2.0           │ 99.8855056762… │ 0.0095272108… │
││ 3.0           │ 99.479166666… │ 0.12443212703… │ 3.0           │ 99.9191792805… │ 0.0134696962… │
││ 3.0           │ 99.185076395… │ 0.08807262885… │ 3.0           │ 99.8810157775… │ 0.0254992345… │
││ 2.0           │ 98.895473480… │ 0.74292283587… │ 2.0           │ 99.8720359802… │ 0.0285762376… │
││ 3.0           │ 98.917922973… │ 0.12731060009… │ 3.0           │ 99.7755025227… │ 0.0339000127… │
││ 3.0           │ 99.160380045… │ 0.10462736537… │ 3.0           │ 99.8742802937… │ 0.0338959706… │
││ 3.0           │ 99.461207071… │ 0.14062994758… │ 3.0           │ 99.8832600911… │ 0.0509967896… │
││ 2.0           │ 97.836296081… │ 0.73593657426… │ 2.0           │ 99.6767234802… │ 0.0190490268… │
└┴───────────────┴───────────────┴────────────────┴───────────────┴────────────────┴───────────────┘